In [1]:
import pandas as pd
import os
import sys
import json
import numpy as np
import tensorflow as tf
import keras.layers as L
import keras.activations as A
import keras.optimizers as O
import keras.losses as Loss
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras import backend as KB
from keras.callbacks import Callback
import time
from fc_conv import FC_CONV

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
ROOTDIR=os.path.abspath("../../")
sys.path.append(ROOTDIR)

In [3]:
from Preprocessing.Preprocessor import preprocessor
from data_utils.keras_generator import Generator
from utils.losses import weighted_cross_entropy_loss,f1_score_metric
from utils.callbacks import get_tensorboard_callback,get_checkpoint_call_back
from utils.tools import construct_model_from_csv
from models.BaseModel import Base_Model

In [4]:
label_encoder_dir=os.path.join(ROOTDIR,"Preprocessing")
data_dir=os.path.join(ROOTDIR,"Data")

In [5]:
files=os.listdir(data_dir)
array=pd.read_hdf(os.path.join(data_dir,files[0]))

P=preprocessor(label_encoder_dir)
array=P.process_batch(array)
action_col=array.columns=='action'

p_config={'labels':6,
        'weightage':[1,1,1,1,1,1],
        'data_directory':data_dir,
        'file_batch_size':1,
        'format_':'h5',
        'ex_batch_size':50,
        'file_examples':5000,
        'max_queue_size':4,
        'Preprocessor':P,
        'static_weightage':False,
         'action_col':action_col}

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
class FC_CONV_Generator(Generator):
    def __init__(self,config):
        super().__init__(config)
    
    def __getitem__(self, idx):
        x,targets=super().__getitem__(idx)
        ob_net=(x[:,2:467]).reshape(-1,155,3,1)
        num_cols=x.shape[1]
#         print (num_cols)
        scalar_features_col_mask=np.arange(num_cols)
        scalar_features_col_mask=(scalar_features_col_mask<2 ) |(scalar_features_col_mask>=467 )
        scalar_features=x[:,scalar_features_col_mask]
        
        return [ob_net,scalar_features],targets

In [7]:
G=FC_CONV_Generator(config=p_config)

In [8]:
from multiprocessing import Process
p1 = Process(target=G.generate)
p1.start()

/home/armughan/anaconda3/envs/py3.6/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
input_shape=[(155,3,1),(7,)]
num_outputs=p_config['labels']
optimizer=O.Adam(lr=0.0001)
design_config_files=["conv_net.csv","fc_net.csv","merged_net.csv"]
model=FC_CONV(input_shape=input_shape,num_outputs=num_outputs,
        optimizer=optimizer,arch_config_files=design_config_files)
model.construct_model()

/home/armughan/Desktop/side projects/aai/cloned/development/new/internship.ai/models/fc_conv/fc_conv.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  self.Model=Model(inputs=[conv_input,scalar_features],output=logits)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 155, 3, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 155, 3, 64)   640         input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 155, 3, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
dense_1 (D

In [ ]:
model.train_generator(G)

Epoch 1/10
 623/1000 [=================>............] - ETA: 7:49 - loss: 0.2349 - acc: 0.8888 - f1_calc: 0.4394